In [5]:
import os, glob
import numpy as np
import trimesh
from scipy.spatial import cKDTree
from sklearn.manifold import TSNE

# ─── adjust to your notebook’s cwd ───
chairs_dir = "embeddings/chairs"
out_dir    = os.path.join(chairs_dir, "colored")
os.makedirs(out_dir, exist_ok=True)

# 1) find your .npz files
pattern   = os.path.join(chairs_dir, "*_pc2048_tokens.npz")
npz_files = sorted(glob.glob(pattern))
print("Found .npz files:", npz_files)

all_embs, all_ctrs, meshes, ids = [], [], [], []
for npz_path in npz_files:
    print("\nLoading:", npz_path)
    data = np.load(npz_path)
    print("  keys in archive:", data.files)
    tokens  = data["tokens"]   # shape (512,384)
    centers = data["centers"]  # shape (512,  3)

    base     = os.path.basename(npz_path).split("_")[0]
    short_id = base[:6]
    ids.append(short_id)

    all_embs.append(tokens)
    all_ctrs.append(centers)

    # load mesh; if it's a Scene, merge into one Trimesh
    obj_path = os.path.join(chairs_dir, f"{short_id}.obj")
    raw = trimesh.load(obj_path, process=False)
    if isinstance(raw, trimesh.Scene):
        # merge all sub-meshes into one
        raw = trimesh.util.concatenate(tuple(raw.geometry.values()))
        print("  merged Scene into one mesh")
    meshes.append(raw)

# sanity check
if not all_embs:
    raise RuntimeError(f"No embeddings loaded! Check {pattern}.")

# 2) joint t-SNE → 3D
stacked = np.vstack(all_embs)
tsne    = TSNE(n_components=3, perplexity=30, random_state=42)
E3      = tsne.fit_transform(stacked)
counts  = [e.shape[0] for e in all_embs]
splits  = np.cumsum(counts)[:-1]
E3_list = np.split(E3, splits, axis=0)

def normalize_rgb(X):
    mn, mx = X.min(0), X.max(0)
    return (X - mn) / (mx - mn)

# 3) bake & export
for short_id, mesh, ctrs, e3 in zip(ids, meshes, all_ctrs, E3_list):
    rgb_pts = normalize_rgb(e3)   # (512×3) in [0,1]

    # nearest-vertex lookup
    tree = cKDTree(mesh.vertices)
    _, vidx = tree.query(ctrs)

    # accumulate + average
    vc = np.zeros((len(mesh.vertices), 3))
    ct = np.zeros(len(mesh.vertices))
    for i, v in enumerate(vidx):
        vc[v] += rgb_pts[i]
        ct[v] += 1
    mask = ct > 0
    vc[mask] /= ct[mask, None]

    mesh.visual.vertex_colors = (vc * 255).astype(np.uint8)
    out_ply = os.path.join(out_dir, f"{short_id}_tsneRGB.ply")
    mesh.export(out_ply)
    print(f"→ wrote {out_ply}")


Found .npz files: ['embeddings/chairs/1006be65e7bc937e9141f9b58470d646_pc2048_tokens.npz', 'embeddings/chairs/1015e71a0d21b127de03ab2a27ba7531_pc2048_tokens.npz', 'embeddings/chairs/107caefdad02cf1c8ab8e68cb52baa6a_pc2048_tokens.npz', 'embeddings/chairs/10d5c2f88b60bbf5febad4f49b26ec52_pc2048_tokens.npz']

Loading: embeddings/chairs/1006be65e7bc937e9141f9b58470d646_pc2048_tokens.npz
  keys in archive: ['centers', 'tokens']
  merged Scene into one mesh

Loading: embeddings/chairs/1015e71a0d21b127de03ab2a27ba7531_pc2048_tokens.npz
  keys in archive: ['centers', 'tokens']
  merged Scene into one mesh

Loading: embeddings/chairs/107caefdad02cf1c8ab8e68cb52baa6a_pc2048_tokens.npz
  keys in archive: ['centers', 'tokens']

Loading: embeddings/chairs/10d5c2f88b60bbf5febad4f49b26ec52_pc2048_tokens.npz
  keys in archive: ['centers', 'tokens']
  merged Scene into one mesh
→ wrote embeddings/chairs/colored/1006be_tsneRGB.ply
→ wrote embeddings/chairs/colored/1015e7_tsneRGB.ply
→ wrote embeddings/c

In [2]:
import os, glob
# Try common variants:
print(glob.glob("embeddings/chairs/colored/*.ply"))
print(glob.glob("notebooks/embeddings/chairs/colored/*.ply"))
print(glob.glob("../embeddings/chairs/colored/*.ply"))


['embeddings/chairs/colored/107cae_tsneRGB.ply', 'embeddings/chairs/colored/10d5c2_tsneRGB.ply', 'embeddings/chairs/colored/1006be_tsneRGB.ply', 'embeddings/chairs/colored/1015e7_tsneRGB.ply']
[]
[]


In [1]:
# ─── Simplified t-SNE + color-on-mesh visualization ───

import os, glob
import numpy as np
import trimesh
from scipy.spatial import cKDTree
from sklearn.manifold import TSNE
import plotly.graph_objects as go

# 1) Point to your data folder
chairs_dir = "embeddings/chairs"
npz_paths  = sorted(glob.glob(os.path.join(chairs_dir, "*_pc2048_tokens.npz")))

# 2) Load all embeddings, centers, and meshes
ids, all_embs, all_ctrs, meshes = [], [], [], []
for npz in npz_paths:
    data = np.load(npz)
    tokens, centers = data["tokens"], data["centers"]
    short_id = os.path.basename(npz).split("_")[0][:6]
    
    raw = trimesh.load(os.path.join(chairs_dir, f"{short_id}.obj"), process=False)
    if isinstance(raw, trimesh.Scene):
        raw = trimesh.util.concatenate(tuple(raw.geometry.values()))
    
    ids.append(short_id)
    all_embs.append(tokens)
    all_ctrs.append(centers)
    meshes.append(raw)

# 3) Joint 3D t-SNE across all tokens
stacked = np.vstack(all_embs)
E3 = TSNE(n_components=3, perplexity=30, random_state=42).fit_transform(stacked)
splits = np.cumsum([e.shape[0] for e in all_embs])[:-1]
E3_list = np.split(E3, splits, axis=0)

# Helper for min–max normalization
def normalize_rgb(X):
    mn, mx = X.min(axis=0), X.max(axis=0)
    return (X - mn) / (mx - mn)

# 4) Assign colors and plot inline
for short_id, mesh, centers, e3 in zip(ids, meshes, all_ctrs, E3_list):
    # normalize to [0,1]
    rgb_pts = normalize_rgb(e3)

    tree = cKDTree(centers)
    K = 5  # or 10
    weights = np.empty((len(mesh.vertices), K))
    indices = np.empty((len(mesh.vertices), K), dtype=int)

    # query k nearest centers for every vertex
    dists, idxs = tree.query(mesh.vertices, k=K)
    # build inverse‐distance weights
    w = 1.0/(dists + 1e-6)
    w /= w.sum(axis=1, keepdims=True)

    # compute per-vertex colour
    vc = (rgb_pts[idxs] * w[...,None]).sum(axis=1)

    # bake and plot exactly as before:
    mesh.visual.vertex_colors = (vc*255).astype(np.uint8)

    # extract for Plotly
    x, y, z = mesh.vertices.T
    i, j, k = mesh.faces.T
    colors   = mesh.visual.vertex_colors[:, :3] / 255.0

    fig = go.Figure(
        data=go.Mesh3d(
            x=x, y=y, z=z,
            i=i, j=j, k=k,
            vertexcolor=colors,
            flatshading=True
        )
    )
    fig.update_layout(
        title=f"t-SNE Colored Mesh: {short_id}",
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        ),
        margin=dict(l=0, r=0, t=30, b=0)
    )
    fig.show()


In [19]:
!pip install --upgrade kaleido

In [2]:

import os, shutil
import numpy as np
import imageio
import plotly.graph_objects as go

# where to put your gifs & temp frames
OUT_DIR    = "gifs"
TEMP_DIR   = "gifs/_frames"
os.makedirs(OUT_DIR, exist_ok=True)
if os.path.exists(TEMP_DIR):
    shutil.rmtree(TEMP_DIR)
os.makedirs(TEMP_DIR)

# parameters
N_FRAMES = 60            # more frames → smoother spin
RADIUS   = 2.5           # distance of camera eye
UP        = dict(x=0, y=0, z=1)

# helper to make camera at angle theta (radians)
def camera_at(theta):
    return dict(
        eye=dict(
            x=RADIUS * np.cos(theta),
            y=RADIUS * np.sin(theta),
            z=RADIUS * 0.2   # slight elevation
        ),
        up=UP
    )

for short_id, mesh in zip(ids, meshes):
    # extract geometry & colors
    x,y,z          = mesh.vertices.T
    i,j,k          = mesh.faces.T
    colors         = mesh.visual.vertex_colors[:, :3] / 255.0
    
    # build a static Figure
    fig = go.Figure(
        data=go.Mesh3d(
            x=x, y=y, z=z,
            i=i, j=j, k=k,
            vertexcolor=colors,
            flatshading=True
        )
    )
    fig.update_layout(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
        ),
        margin=dict(l=0, r=0, t=0, b=0)
    )

    # 1) render each frame
    frame_paths = []
    for f in range(N_FRAMES):
        theta = 2*np.pi * f / N_FRAMES
        fig.update_layout(scene_camera=camera_at(theta))
        fname = os.path.join(TEMP_DIR, f"{short_id}_{f:03d}.png")
        fig.write_image(fname)        # requires kaleido
        frame_paths.append(fname)

    # 2) stitch into a gif
    images = [imageio.v2.imread(fp) for fp in frame_paths]
    out_gif = os.path.join(OUT_DIR, f"{short_id}.gif")
    imageio.mimsave(out_gif, images, fps=20)  # adjust fps if you like

    print(f"→ wrote {out_gif}")

# clean up temp frames if you like
shutil.rmtree(TEMP_DIR)

print("Done! Your GIFs are in", OUT_DIR)


→ wrote gifs/1006be.gif
→ wrote gifs/1015e7.gif
→ wrote gifs/107cae.gif
→ wrote gifs/10d5c2.gif
Done! Your GIFs are in gifs
